In [3]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/tatabahasabm.tripod.com/quiz-tatabahasa.jsonl

In [4]:
from tqdm import tqdm
import os
import json
import random

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained('mesolitica/mallam-1.1B-4096')
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/mallam-1.1B-4096', 
    use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="cuda:0"
)

In [6]:
questions = []
with open('quiz-tatabahasa.jsonl') as fopen:
    for no, l in enumerate(fopen):
        l = json.loads(l)
        soalan = [l['question']]
        jawapan = None
        for c, k in l['choices'].items():
            soalan.append(f"{c}. {k['text']}")
            if k['answer']:
                jawapan = c
        
        data = {
            'no': no,
            'objektif': 'Jawab soalan yang diberikan' if l['instruction'] is None else l['instruction'],
            'soalan': '\n'.join(soalan),
            'jawapan': jawapan,
        }
        questions.append(data)
len(questions)

349

In [7]:
arange = set(range(len(questions)))

In [8]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

In [9]:
i = 2
prompts = []

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts).strip()
print(prompt)

objektif: Jawab soalan yang diberikan
soalan: ............. , bisanya sengatan tebuan itu !
A. Wah
B. Nah
C. Eh
D. Aduh
jawapan:


In [10]:
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

{'input_ids': tensor([[12184,    29, 17797,  5075,   313,  3107,   202,  7808,    29,  3797,
          7531,  7531,   714,  1056,  9881,   605,  9454,   442, 11682,   482,
           508,  8925,   202,    36,    17, 11031,   202,    37,    17, 25031,
           202,    38,    17,  9222,   202,    39,    17,  1771,   454,   202,
            77,  4255,    29]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [ ]:
for i in tqdm(range(len(questions))):
    
    prompts = []
    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=3,
                top_p=0.95,
                top_k=50,
                temperature=0.1,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('jawapan:')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').replace('(', '').replace(')', '').upper().strip().split()
            splitted = [t for t in splitted if len(t) == 1 and t in {'A', 'B', 'C', 'D'}]
            repeat.append(splitted[0])
    
        except Exception as e:
            print(e)
            pass
    
    questions[i]['output'] = repeat


  0%|          | 0/349 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  0%|          | 1/349 [00:00<01:46,  3.26it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


list index out of range
list index out of range
list index out of range
list index out of range



  1%|          | 2/349 [00:00<01:46,  3.27it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


list index out of range
list index out of range
list index out of range
list index out of range


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  1%|          | 3/349 [00:00<01:44,  3.31it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


list index out of range
list index out of range
list index out of range
list index out of range



 86%|████████▌ | 299/349 [01:31<00:15,  3.26it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


list index out of range


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

 86%|████████▌ | 300/349 [01:31<00:15,  3.26it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [24]:
def most_common(l):
    return max(set(l), key=l.count)

In [25]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

20.8955223880597